In [1]:
# Импортируем необходимые библиотеки:
# pandas для манипуляции данными,
# scikit-surprise для функциональности рекомендательных систем, такой как чтение датасетов и применение алгоритмов.
import pandas as pd
from surprise import Dataset, Reader, SVDpp

# Загружаем датасет с рейтингами фильмов.
ratings = pd.read_csv("ratings.csv")

# Создаем таблицу сводки, чтобы увидеть рейтинги, которые пользователи ставят каждому фильму.
# Индекс: userId, Колонки: movieId, Значения: рейтинг.
user_movie_rating = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# Показываем первые 100 строк для быстрого просмотра.
user_movie_rating.head(100)  

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Определяем объект Reader, указывая шкалу рейтинга.
reader = Reader(rating_scale=(0.5, 5))  # Рейтинги фильмов варьируются от 0.5 до 5.

# Загружаем данные из DataFrame.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Строим полный тренировочный набор данных.
trainset = data.build_full_trainset()

# Создаем экземпляр алгоритма SVD++.
algorithm = SVDpp()
 # Обучаем модель на тренировочном наборе.
algorithm.fit(trainset) 

# Подготавливаем пустой список для тестового набора данных.
testset = []

# Проходим по всем пользователям и фильмам для создания тестового набора.
for uid in trainset.all_users():
    for iid in trainset.all_items():
        # Проверяем отсутствие пары (uid, iid) в тренировочном наборе.
        if not trainset.ur.get(uid) or (iid not in map(lambda x:x[0], trainset.ur[uid])):
            # Добавляем в тестовый набор с None в качестве фактического рейтинга.
            testset.append((trainset.to_raw_uid(uid), trainset.to_raw_iid(iid), None))

# Получаем предсказания для отсутствующих значений.
predictions = algorithm.test(testset)

# Подготавливаем список для хранения предсказанных рейтингов.
pred_ratings = []
for pred in predictions:
    pred_ratings.append([pred.uid, pred.iid, pred.est])

In [3]:
# Преобразуем список предсказаний в DataFrame.
pred_df = pd.DataFrame(pred_ratings, columns=['userId', 'movieId', 'rating'])

# Объединяем предсказанные рейтинги с исходными рейтингами.
complete_ratings = pd.concat([ratings[['userId', 'movieId', 'rating']], pred_df])

# Создаем итоговую таблицу сводки с полными данными, включающую как фактические, так и предсказанные рейтинги.
complete_user_movie_rating = complete_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Сортируем столбцы итоговой сводной таблицы согласно исходной таблице сводки рейтингов.
complete_user_movie_rating = complete_user_movie_rating[user_movie_rating.columns]

# Показываем первые 100 строк итоговой таблицы для быстрого просмотра.
complete_user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.294113,4.000000,3.554711,3.598930,4.000000,3.709089,3.619297,3.822295,4.138207,...,4.241959,3.971497,4.121876,4.232585,4.170376,4.245724,4.046875,4.129473,4.081407,4.370713
2,3.858331,3.644491,3.421590,2.902929,3.102959,4.026015,3.285115,3.098073,3.064605,3.743408,...,3.519022,3.375125,3.575342,3.636100,3.594380,3.724546,3.604921,3.526162,3.583180,3.675628
3,2.576129,2.839697,2.254433,1.835107,1.835418,2.730060,1.551390,2.261575,2.455056,2.744248,...,2.488245,2.809733,2.868686,2.738129,2.404417,2.564213,2.559446,2.754940,2.427256,2.847397
4,4.138230,3.734429,3.595299,2.798733,3.219246,3.594203,3.290205,2.915631,2.850871,3.608178,...,3.532573,3.213621,3.451785,3.677157,3.313326,3.489921,3.158277,3.406072,3.480564,3.227023
5,4.000000,3.599387,3.163167,2.886144,2.914275,3.888983,3.116139,3.068276,3.252838,3.461716,...,3.523673,3.434690,3.524693,3.558114,3.442634,3.522899,3.543623,3.423027,3.525973,3.496300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.000000,3.571379,3.579387,2.768828,2.885829,4.181017,3.147599,3.198299,3.044277,3.110887,...,3.707639,3.388223,3.565226,3.463845,3.546151,3.411468,3.562743,3.464195,3.358771,3.696921
97,4.578149,4.113185,3.743825,3.491871,3.508286,4.458262,3.690605,3.715776,3.672426,4.109321,...,4.075238,4.065976,4.113050,4.222614,4.033256,4.032014,4.139308,4.104968,4.056563,4.125243
98,4.500000,3.707264,3.628191,3.040535,3.234264,3.976709,3.451282,3.199056,3.241330,3.722728,...,3.736352,3.544826,3.733392,3.923257,3.671304,3.769700,3.637920,3.787715,3.656373,3.847589


In [4]:
# Экспортируем итоговую таблицу в CSV файл.
complete_user_movie_rating.to_csv("predicted_ratings.csv")